In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv',index_col=0)
test = pd.read_csv('data/test.csv')

In [3]:
train.isna().sum()

cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
cont14    0
target    0
dtype: int64

In [4]:
test.isna().sum()

id        0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
cont14    0
dtype: int64

In [5]:
X_train,X_val,y_train,y_val = train_test_split(train.drop('target',axis=1),train['target'],test_size=0.2)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((240000, 14), (60000, 14), (240000,), (60000,))

In [ ]:
def rf_objective(trial):
    rf_params = {
      'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
      'max_depth': trial.suggest_int('max_depth', 2, 32),
      'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
      'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
      'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
      'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    rf_model=RandomForestRegressor(**rf_params)    
    rf_model.fit(X_train,y_train)

    rf_rmse = np.sqrt(mean_squared_error(y_val,rf_model.predict(X_val)))
    return rf_rmse

rf_study = optuna.create_study(direction='minimize')
rf_study.optimize(rf_objective, n_trials=50,n_jobs=-1)

[I 2024-04-09 00:36:06,379] A new study created in memory with name: no-name-c076bb60-96cb-4fa2-af5d-fab82f46e346
